# Intro To QDrant

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, TypeAlias, Union
import logging
import warnings

# Standard imports
import numpy as np
from pprint import pprint
import pandas as pd
import polars as pl
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

## Import Dependencies

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
ENCODER_CHECKPOINT: str = "all-MiniLM-L6-v2"

encoder = SentenceTransformer(ENCODER_CHECKPOINT)

In [4]:
# Add datasets

documents: list[str, Any] = [
    {
        "name": "The Time Machine",
        "description": "A man travels through time and witnesses the evolution of humanity.",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Ender's Game",
        "description": "A young boy is trained to become a military leader in a war against an alien race.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Brave New World",
        "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Dune",
        "description": "A desert planet is the site of political intrigue and power struggles.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Foundation",
        "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Snow Crash",
        "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "The War of the Worlds",
        "description": "A Martian invasion of Earth throws humanity into chaos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "The Hunger Games",
        "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "The Andromeda Strain",
        "description": "A deadly virus from outer space threatens to wipe out humanity.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "The Three-Body Problem",
        "description": "Humans encounter an alien civilization that lives in a dying system.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]

In [5]:
# Define Storage Location
URL: str = "http://localhost:6333"
# qdrant = QdrantClient(":memory:")
qdrant = QdrantClient(URL)
INDEX_NAME: str = "my_books"

# Create a collection
# Use recreate_collection if you are experimenting and running the script several times.
# This function will first try to remove an existing collection with the same name.
qdrant.recreate_collection(
    collection_name=INDEX_NAME,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [6]:
# vectors: list[float] =

# Upload data to collection
qdrant.upload_records(
    collection_name=INDEX_NAME,
    records=[
        models.Record(
            id=idx, vector=encoder.encode(doc["description"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

### Query The Database

In [7]:
query: str = "alien invasion"
query_vector: list[float] = encoder.encode(query).tolist()
N: int = 3

hits: list[Any] = qdrant.search(
    collection_name=INDEX_NAME,
    query_vector=query_vector,
    limit=N,
)

for hit in hits:
    print(hit.payload, "score:", hit.score)

{
    'author': 'H.G. Wells',
    'description': 'A Martian invasion of Earth throws humanity into chaos.',
    'name': 'The War of the Worlds',
    'year': 1898
}
score: 0.5700934

{
    'author': 'Douglas Adams',
    'description': 'A comedic science fiction series following the misadventures of an unwitting human and his 
alien friend.',
    'name': "The Hitchhiker's Guide to the Galaxy",
    'year': 1979
}
score: 0.50404686

{
    'author': 'Liu Cixin',
    'description': 'Humans encounter an alien civilization that lives in a dying system.',
    'name': 'The Three-Body Problem',
    'year': 2008
}
score: 0.45902944

### Narrow down the query

- How about the most recent book from the early 2000s?

In [8]:
query: str = "alien invasion"
query_vector: list[float] = encoder.encode(query).tolist()
query_filter: models.Filter = models.Filter(
    must=[models.FieldCondition(key="year", range=models.Range(gte=2000))]
)


hits: list[Any] = qdrant.search(
    collection_name="my_books",
    query_vector=query_vector,
    query_filter=query_filter,
    limit=N,
)
for hit in hits:
    print(hit.payload, "score:", hit.score)

{
    'author': 'Liu Cixin',
    'description': 'Humans encounter an alien civilization that lives in a dying system.',
    'name': 'The Three-Body Problem',
    'year': 2008
}
score: 0.45902944

{
    'author': 'Suzanne Collins',
    'description': 'A dystopian society where teenagers are forced to fight to the death in a televised 
spectacle.',
    'name': 'The Hunger Games',
    'year': 2008
}
score: 0.16074508

# Neural Search Service

In [9]:
from pathlib import Path

from datasets import Dataset, load_dataset
from tqdm.notebook import tqdm

In [10]:
data_fp: Path = Path("../../data")
fp: Path = data_fp / "startups_demo.json"
data_files: dict[str, Any] = {"train": str(fp)}

startup_data: Dataset = load_dataset(path="json", data_files=data_files)
startup_data

DatasetDict({
    train: Dataset({
        features: ['name', 'images', 'alt', 'description', 'link', 'city'],
        num_rows: 40474
    })
})

In [11]:
print(startup_data.get("train")[:3])

{
    'name': ['SaferCodes', 'Human Practice', 'StyleSeek'],
    'images': [
        'https://safer.codes/img/brand/logo-icon.png',
        'https://d1qb2nb5cznatu.cloudfront.net/startups/i/373036-94d1e190f12f2c919c3566ecaecbda68-thumb_jpg.jpg?bus
ter=1396498835',
        'https://d1qb2nb5cznatu.cloudfront.net/startups/i/3747-bb0338d641617b54f5234a1d3bfc6fd0-thumb_jpg.jpg?buste
r=1329158692'
    ],
    'alt': [
        'SaferCodes Logo QR codes generator system forms for COVID-19',
        'Human Practice -  health care information technology',
        'StyleSeek -  e-commerce fashion mass customization online shopping'
    ],
    'description': [
        'QR codes systems for COVID-19.\nSimple tools for bars, restaurants, offices, and other small proximity 
businesses.',
        'Point-of-care word of mouth\nPreferral is a mobile platform that channels physicians’ interest in 
networking with their peers to build referrals within a hospital system.\nHospitals are in a race to employ 
physicians, even though they lose billions each year ($40B in 2014) on employment. Why ...',
        'Personalized e-commerce for lifestyle products\nStyleSeek is a personalized e-commerce site for lifestyle 
products.\nIt works across the style spectrum by enabling users (both men and women) to create and refine their 
unique StyleDNA.\nStyleSeek also promotes new products via its email newsletter, 100% personalized ...'
    ],
    'link': ['https://safer.codes', 'http://humanpractice.com', 'http://styleseek.com'],
    'city': ['Chicago', 'Chicago', 'Chicago']
}

In [12]:
ENCODER_CHECKPOINT: str = "all-MiniLM-L6-v2"

encoder = SentenceTransformer(ENCODER_CHECKPOINT)

In [13]:
encoder.encode(query).tolist()

[0.008490558713674545,
 0.0070844413712620735,
 0.012432715855538845,
 0.016528213396668434,
 0.018315991386771202,
 -0.05017716437578201,
 -0.01185440830886364,
 -0.036697763949632645,
 0.015199929475784302,
 -0.011771843768656254,
 0.02491699904203415,
 -0.03999173268675804,
 0.027960997074842453,
 0.010655614547431469,
 -0.02657207101583481,
 0.035112686455249786,
 0.014779333956539631,
 -0.07972410321235657,
 -0.021407857537269592,
 -0.004412065725773573,
 -0.08960099518299103,
 0.05520902946591377,
 0.011839141137897968,
 0.03228887543082237,
 -0.01812012866139412,
 0.021024376153945923,
 0.021151550114154816,
 0.05961310490965843,
 -0.05800142511725426,
 -0.07097145169973373,
 -0.0015109328087419271,
 0.08469453454017639,
 -0.005802256055176258,
 0.0498209185898304,
 -0.006269002798944712,
 -0.027293283492326736,
 0.02292904444038868,
 -0.09682813286781311,
 0.06549759209156036,
 -0.009998810477554798,
 -0.022183040156960487,
 -0.09672120958566666,
 0.05387067049741745,
 0.004859

In [14]:
def encode_data(example: dict[str, Any]) -> dict[str, Any]:
    """This is used to encode the data using the encoding model."""
    example["vector"] = encoder.encode(example.get("description")).tolist()
    return example

In [15]:
# Encode the description
def encode_data(example: dict[str, Any]) -> dict[str, Any]:
    """This is used to encode the data using the encoding model.
    Parallelised version.
    """
    return {"vector": [encoder.encode(x).tolist() for x in example.get("description")]}

In [16]:
A: Dataset = startup_data.get("train").shuffle(123).select(range(10))

A

Dataset({
    features: ['name', 'images', 'alt', 'description', 'link', 'city'],
    num_rows: 10
})

In [17]:
# Test the function
A = A.map(encode_data, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [18]:
RUN_CELL: bool = False

if RUN_CELL:
    # Add embeddings to the data
    startup_data_emb: Dataset = startup_data.map(encode_data, batched=True)
    startup_data_emb

###  Save Normal Dataset object

```python
startup_data_emb.to_json(f"{fp}.jsonl")
```

In [19]:
# Save data
if RUN_CELL:
    fp: Path = data_fp / "startup_data_enb"

    # Iterate and save because it's a DatasetDict object
    for _split, _dataset in startup_data_emb.items():
        _dataset.to_json(f"{fp}-{_split}.jsonl")

<hr><br>

### Save The Vectors

- DataFrame Approach
  
```python
df: pd.DataFrame = pd.read_json("./startups_demo.json", lines=True)

vectors = encoder.encode(
    [row.alt + ". " + row.description for row in df.itertuples()],
    show_progress_bar=True,
)

print(vectors.shape)

# Download the saved vectors into a new file named startup_vectors.npy
np.save("startup_vectors.npy", vectors, allow_pickle=False)
```

In [21]:
from qdrant_client.models import VectorParams, Distance


# Define Storage Location
URL: str = "http://localhost:6333"
qdrant = QdrantClient(URL)
INDEX_NAME: str = "startups"
RUN_CELL: bool = True

if RUN_CELL:
    # Create a collection
    # Use recreate_collection if you are experimenting and running the script several times.
    # This function will first try to remove an existing collection with the same name.
    qdrant.create_collection(
        collection_name=INDEX_NAME,
        vectors_config=VectorParams(
            size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
            distance=Distance.COSINE,
        ),
    )

### Load Data


In [22]:
from datasets import Dataset, DatasetDict, load_dataset

fp: Path = data_fp / "startup_data_enb-train.jsonl"

emb_data: DatasetDict = load_dataset("json", data_files=str(fp))
emb_data

DatasetDict({
    train: Dataset({
        features: ['name', 'images', 'alt', 'description', 'link', 'city', 'vector'],
        num_rows: 40474
    })
})

In [23]:
print(emb_data.get("train")[10])

{
    'name': 'Rocketmiles',
    'images': 
'https://d1qb2nb5cznatu.cloudfront.net/startups/i/158571-e53ddffe9fb3ed5e57080db7134117d0-thumb_jpg.jpg?buster=1361
371304',
    'alt': 'Rocketmiles -  e-commerce online travel loyalty programs hotels',
    'description': "Fueling more vacations\nWe enable our customers to travel more, travel better and travel 
further. 20M+ consumers stock away miles & points to satisfy their wanderlust.\nFlying around or using credit cards
are the only good ways to fill the stockpile today. We've built the third way. Customers ...",
    'link': 'http://www.Rocketmiles.com',
    'city': 'Chicago',
    'vector': [
        0.0486195944,
        0.0029633229,
        0.0347133689,
        0.061903365,
        0.0022507736,
        0.0771941692,
        0.0784552097,
        0.002627562,
        -0.0285524167,
        -0.0575823449,
        0.0731771886,
        0.0465080738,
        -0.0668561235,
        0.0295296144,
        0.1004876792,
        -0.0398686603,
        0.0562053807,
        -0.0062186783,
        -0.0316168927,
        -0.0259011872,
        -0.0628041402,
        -0.1036843508,
        -0.0405822881,
        0.091833353,
        0.0127129685,
        0.002218978,
        -0.0505007021,
        0.0078837285,
        -0.0213271622,
        -0.0268883333,
        0.0165126212,
        0.0762796924,
        0.0014689385,
        0.0035828194,
        -0.0216542408,
        0.0514446013,
        -0.0380847827,
        -2.09375e-05,
        0.0208412688,
        0.0268358681,
        0.0080078719,
        -0.0371401645,
        0.0272598676,
        0.0334198251,
        -0.0004661346,
        -0.0589151122,
        0.0089077298,
        0.0891029611,
        0.1136547476,
        0.0981594399,
        -0.0220549591,
        0.0149720265,
        -0.0464704968,
        -0.0727069452,
        -0.0430655815,
        -0.008849171,
        0.0200289227,
        -0.0246555265,
        -0.0474199653,
        -0.0316180438,
        0.0267510489,
        -0.0747529343,
        0.0100884689,
        0.0383560397,
        0.0039366279,
        -0.0916233808,
        -0.055781085,
        0.0629381016,
        -0.0523786806,
        0.0326409005,
        -0.0036961597,
        -0.0112250149,
        -0.1012003571,
        0.0428457558,
        -0.035491813,
        0.0173729118,
        0.0402427614,
        0.0028204273,
        0.0529464483,
        0.0154145528,
        0.0106165996,
        -0.0074739563,
        -0.0169292726,
        0.0028575549,
        -0.0519486144,
        -0.0529404432,
        -0.0007137028,
        -0.01769308,
        -0.004647376,
        -0.0535041504,
        0.0702090412,
        0.01162758,
        0.0200581681,
        0.0179416332,
        -0.1056229025,
        0.0103908172,
        -0.1128210723,
        -0.0281571876,
        -0.026682416,
        0.0235353336,
        0.0881875604,
        0.1319362521,
        0.051724676,
        -0.0042191315,
        -0.0479211658,
        -0.0128493831,
        0.0105355503,
        0.0541654155,
        0.0497060269,
        -0.0048995442,
        -0.0570117906,
        0.071547389,
        0.0135492859,
        -0.0461413711,
        -0.0564850792,
        0.0608757623,
        -0.0410831794,
        -0.0077135926,
        0.0659782439,
        0.0844860673,
        -0.0034544405,
        0.0104193138,
        -0.0007293495,
        -0.0282565057,
        -0.0631003752,
        -0.0416286066,
        0.0659256056,
        1.16731842e-33,
        -0.0750155449,
        0.0434611551,
        0.0588933304,
        0.0973147526,
        -0.0120255658,
        -0.0099171223,
        -0.0915798396,
        -0.0148445414,
        -0.0318260677,
        -0.019266719,
        0.0073951851,
        -0.0223129783,
        0.0633367524,
        0.0648519471,
        0.0563275367,
        -0.0975330025,
        -0.0651825741,
        0.0101502398,
        0.1082893014,
        -0.0565191321,
        0.0608095676,
        -0.14204

In [24]:
print(emb_data.get("train").select([3, 4]).remove_columns("vector").to_list())

[
    {
        'name': 'Scout',
        'images': 
'https://d1qb2nb5cznatu.cloudfront.net/startups/i/190790-dbe27fe8cda0614d644431f853b64e8f-thumb_jpg.jpg?buster=1389
652078',
        'alt': 'Scout -  security consumer electronics internet of things',
        'description': "Hassle-free Home Security\nScout is a self-installed, wireless home security system. We've 
created a more open, affordable and modern system than what is available on the market today. With month-to-month 
contracts and portable devices, Scout is a renter-friendly solution for the other ...",
        'link': 'http://www.scoutalarm.com',
        'city': 'Chicago'
    },
    {
        'name': 'Invitation codes',
        'images': 'https://invitation.codes/img/inv-brand-fb3.png',
        'alt': 'Invitation App - Share referral codes community ',
        'description': 'The referral community\nInvitation App is a social network where people post their referral
codes and collect rewards on autopilot.',
        'link': 'https://invitation.codes',
        'city': 'Chicago'
    }
]

In [25]:
if RUN_CELL:
    vectors_dict: list[dict[str, list[float]]] = (
        emb_data.get("train").select_columns("vector").to_list()
    )
    vectors: list[list[float]] = [x.get("vector") for x in vectors_dict]
    payload: list[dict[str, Any]] = (
        emb_data.get("train").remove_columns("vector").to_list()
    )

In [26]:
if RUN_CELL:
    # Number of vectors that will be uploaded in a single request
    BATCH_SIZE: int = 256

    qdrant.upload_collection(
        collection_name=INDEX_NAME,
        vectors=vectors,
        payload=payload,
        ids=None,  # Vector ids will be assigned automatically
        batch_size=BATCH_SIZE,
    )